In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')

In [ ]:
import numpy as np
from sklearn import model_selection

indexes = np.random.randint(70000, size=7000)

data = mnist.data[indexes]
#print(data.shape)
target = mnist.target[indexes]

xtrain, xtest, ytrain, ytest = model_selection.train_test_split(
    data, target, train_size=0.8
)


In [ ]:
from sklearn import neighbors

k = 10

clf = neighbors.KNeighborsClassifier(k)
clf.fit(xtrain, ytrain)

probas = clf.predict_proba(xtest)

In [ ]:
from sklearn import datasets
import matplotlib.pyplot as plt

itest = 4

images = data.reshape((-1, 28, 28))
plt.imshow(images[itest],cmap=plt.cm.gray_r,interpolation="nearest")
plt.show()

print(probas[itest])

In [ ]:
#clf.score(xtest, ytest)
found_classes = clf.predict(xtest)

In [ ]:
for itest in range(1,20):
    #itest = 4
    
    res =  " OK" if ytest[itest] == found_classes[itest] else ""

    print("Classe : " + ytest[itest] + " / Attendu " + found_classes[itest] + res)

In [ ]:
# BAD
"""
from sklearn.model_selection import KFold

scores_k = [0 for i in range(0,16)]

nb_splits = 10

# k = Nombre de voisins à considérer
for k in range(2,16):
    clf = neighbors.KNeighborsClassifier(k)    
    kf = KFold(n_splits=nb_splits, shuffle=True)

    

    for train_indexes, test_indexes in kf.split(data):
        # Calcul des xtest/ytest/xtrain/ytrain
        xtrain = data[train_indexes]
        xtest = data[test_indexes]
        ytrain = target[train_indexes]
        ytest = target[test_indexes]
    
        clf.fit(xtrain, ytrain)
        
        print(".", end = '')
        scores_k[k] += clf.score(xtest, ytest) / nb_splits
        
    
    print(scores_k[k])
"""

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np
import time

scores_k2 = [0 for i in range(0,16)]
times_train_k = [0 for i in range(0,16)]
times_score_k = [0 for i in range(0,16)]


nb_splits = 10

# k = Nombre de voisins à considérer
for k in range(2,16):
    clf = neighbors.KNeighborsClassifier(k, n_jobs=-1)
    start_time = time.process_time()
    clf.fit(xtrain, ytrain)
    times_train_k[k] = (time.process_time() - start_time)
    
    start_time = time.process_time()
    score = np.mean(cross_val_score(clf, xtrain, ytrain, cv=10))
    times_score_k[k] = (time.process_time() - start_time)
    
    print("K = " + str(k) + " -> " + str(score))
    scores_k2[k] = score


In [ ]:

plt.plot([x for x in range(0, 16)], times_train_k)
plt.plot([x for x in range(0, 16)], times_score_k)

print(times_score_k)

plt.legend(['Temps d\'entrainement', 'Temps de prédiction'], loc='best')
plt.xlabel("k")
plt.ylabel("Temps (secondes)")
plt.title("Temps d'entrainement et de prédiction pour différentes valeurs de k")

#sklearn tsne

In [ ]:

#plt.plot([x for x in range(2, 16)], scores_k[2:])
plt.plot([x for x in range(2, 16)], scores_k2[2:])

#plt.legend(['KFold', 'cross_val_score'], loc='upper right')
plt.xlabel("k")
plt.ylabel("Precision")
plt.title("Score pour différentes valeurs de k")

#sklearn tsne

# Résultat

Pour la suite, on prends la valeur k=5

In [ ]:
from sklearn import neighbors

k=5
rates = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

scores_rate = []
for rate in rates:
    xtrain, xtest, ytrain, ytest = model_selection.train_test_split(
        data, target, train_size=rate
    )

    clf = neighbors.KNeighborsClassifier(k)
    clf.fit(xtrain, ytrain)
    
    #print(str(len(ytrain)) + " - - - " + str(rate))
    score = clf.score(xtest, ytest)
    
    #score = np.mean(cross_val_score(clf, xtest, ytest, cv=10))

    print("Rate = " + str(rate) + " -> " + str(score))
    scores_rate.append(score)    

In [ ]:
plt.plot([x for x in rates], scores_rate)

plt.xlabel("Pourcentage consacré au train")
plt.ylabel("Precision")
plt.title("Score pour différents pourcentages consacrés au train(k=" + str(k) +")")


# Résultat
Plus on alloue de données à l'entrainement, plus l'algorithme est efficace pour la classification.

In [ ]:
from sklearn import neighbors

k=5
rate=0.8

# De 400 à 8000
sizes = [200*i for i in range(1, 51)]

scores_size = []
for size in sizes:
    
    indexes2 = np.random.randint(70000, size=size)

    data2 = mnist.data[indexes2]
    target2 = mnist.target[indexes2]

    xtrain, xtest, ytrain, ytest = model_selection.train_test_split(
        data2, target2, train_size=rate
    )

    clf = neighbors.KNeighborsClassifier(k)
    clf.fit(xtrain, ytrain)
    
    score = clf.score(xtest, ytest)

    print("Size = " + str(size) + " -> " + str(score))
    scores_size.append(score)

In [ ]:
plt.plot([x for x in sizes], scores_size)

plt.xlabel("Taille de l'échantillon")
plt.ylabel("Precision")
plt.title("Score pour différentes tailles d'échantillon")


# Résultat
Plus la taille de l'échantillon est importante, plus la précision de l'algorithme est grande.

In [ ]:
from sklearn import neighbors

k=5
rate = 0.8
ps = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
scores_p = []

for p in ps:
    xtrain, xtest, ytrain, ytest = model_selection.train_test_split(
        data, target, train_size=rate
    )

    clf = neighbors.KNeighborsClassifier(k, p=p, n_jobs=-1)
    clf.fit(xtrain, ytrain)
    
    score = clf.score(xtest, ytest)

    print("P = " + str(p) + " -> " + str(score))
    scores_p.append(score)    

In [ ]:
plt.plot([x for x in ps], scores_p)

plt.xlabel("Valeurs de P")
plt.ylabel("Precision")
plt.title("Score pour différentes valeurs de p(k=" + str(k) +")")


# Résultat

Augmentation de la précision en augmentant la norme utilisée

In [ ]:
%%time

from sklearn import neighbors
import time


k=5
rate = 0.8
p = 7

xtrain, xtest, ytrain, ytest = model_selection.train_test_split(
    data, target, train_size=rate
)

# Mesure avec n_jobs = 1
start_time = time.time()
start_process_time = time.process_time()
clf = neighbors.KNeighborsClassifier(k, p=p, n_jobs = 1)
clf.fit(xtrain, ytrain)
end_time = time.time()
end_process_time = time.process_time()

score = clf.score(xtest, ytest)


print("n_jobs = 1 -> " + str(score) + " time : " + str(end_time - start_time))
print("n_jobs = 1 -> " + str(score) + " process time : " + str(end_process_time - start_process_time))




# Mesure avec n_jobs = -1
start_time = time.time()
start_process_time = time.process_time()

clf = neighbors.KNeighborsClassifier(k, p=p, n_jobs = -1)
clf.fit(xtrain, ytrain)
end_time = time.time()
end_process_time = time.process_time()

score = clf.score(xtest, ytest)



print("n_jobs = -1 -> " + str(score) + " time : " + str(end_time - start_time))
print("n_jobs = -1 -> " + str(score) + " process time : " + str(end_process_time - start_process_time))

